In [1]:
:set -XNoMonadFailDesugaring
:l SortersM
:l Predicates

In [2]:
selectSortM coinCmp [3,2,1] :: [[Int]]

[[3,2,1],[3,1,2],[2,3,1],[2,1,3],[3,2,1],[3,1,2],[1,3,2],[1,2,3]]

In [3]:
import Control.Monad.State
evalStateT (selectSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[1,3,2],[1,2,3]]

In [4]:
import qualified Data.Set.Monad as SetM (toList)
SetM.toList $ selectSortM coinCmp [3,2,1]

: 

In [5]:
minimumNDM :: (MonadPlus m) => [a] -> m a
minimumNDM [x] = return x
minimumNDM (x:xs) = do
    y <- minimumNDM xs
    return x `mplus` return y

minimumNDM [1,2,3] :: [Int]

[1,2,1,3]

In [6]:
minimumNDM :: (MonadPlus m) => [a] -> m a
minimumNDM [x] = return x
minimumNDM (x:xs) = return x `mplus` minimumNDM xs

minimumNDM [1,2,3] :: [Int]

[1,2,3]

In [7]:
import qualified Data.List as List

selectSortNDM :: (Eq a, MonadPlus m) => [a] -> m [a]
selectSortNDM [] = return []
selectSortNDM xs = do
    x <- minimumNDM xs
    fmap (x:) (selectSortNDM (List.delete x xs))

selectSortNDM [1,2,3] :: [[Int]]

[[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]]